<a href="https://colab.research.google.com/github/alexch142/from-colab/blob/main/tips_and_trics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#создание каталогов
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "cats"))
    os.makedirs(os.path.join(dir_name, "dogs"))


In [ ]:
# подключение GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')
# просмотр содержимого папки на диске
! ls /content/drive/My\ Drive/datasets/data_happy
# полный адрес подключенного
filename = '/content/drive/My Drive/datasets/data_happy/train_happy.h5'

In [ ]:
#копирование картинок
def copy_images(start_index, end_index, source_dir_0,source_dir_1, dest_dir):
    for i in range(start_index, end_index):
       
        shutil.copy2(os.path.join(source_dir_0, "cat." + str(i) + ".jpg"), 
                   os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir_1, "dog." + str(i) + ".jpg"), 
                   os.path.join(dest_dir, "dogs"))

In [ ]:
#Добавление плохоразпознаных картинок в обучающую выборку для повторного обучения

выбираем данные в которых сеть не уверена (0.88<) но вместо удаления добавляем их в обучающий набор - как бы увличивая их вес

In [ ]:
def select_x(x_aug,y_aug):
    predictions = model.predict(x_aug)
    err_count = 0
    indexes = np.argmax(prediction,axis=1)
    answers = np.argmax(y_aug,axis=1)
    nums = [0,0,0,0,0,0,0,0,0,0]
    err_count = 0
    for x in range(0,x_aug.shape[0]):
        if (predictions[x,indexes[x]]<0.88) or (indexes[x]!=answers[x]):
            err_count +=1
            nums[answers[x]] +=1
    print("deleted : ", err_count)
    
    ind= 0
    x_sel = np.empty([x_aug.shape[0]+err_count,32,32,3])
    y_sel = np.empty([y_aug.shape[0]+err_count,10])
    for x in range(0,x_aug.shape[0]):
        if (predictions[x,indexes[x]]<0.88)or(indexes[x]!=answers[x]):
            x_sel[ind]=x_aug[x]
            y_sel[ing]=y_aug[x]
            ind+=1
    for x in range(0,x_aug.shape[0]):
        x_sel[ind] = x_aug[x]
        y_sel[ind] = y_aug[x]
        ind+=1
    gc.collect() #освобождение памяти.не факт что надо
    print(nums)
    return(x_sel,y_sel)

#вызываем так: (X_new,Y_new)=select(x_data,y_data)


In [ ]:
#Генератор для подачи картинок в fit с диска без предварительной загрузки в память

In [ ]:
from segmentation_models import Unet
import itertools
import scipy
import numpy as np
import keras.utils
# прочитать пути к файлам можно с помощью glob как в описании
images_paths = ... # массив строк-путей картинок
segmentations_paths = ... # массив строк-путей разметки
#перемешивание базы как в описании
...

In [ ]:
# Для того, чтобы бесконечно брать из последовательности с помощью next:
c = itertools.cycle(zip(images_paths, segmentations_paths))
batch_size = 6
classes_count = 12
def generator():
    while True:
        segmentation_for_batch = []
        images_for_batch = []
        for i in range(batch_size):
            image_path, segmentation_path = next(c)

            # считываем картинку
            image = scipy.misc.imread(image_path)

            #препроцессинг, можно просто перевод во float
            ...

            segmentation = scipy.misc.imread(segmentation_path)

            # ресайзинг image и segmentation в 352, 480
            ...

            # первод из индексов классов segmentation в one-hot вектора segmentation_one_hot
            # для меньшего количества классов - 7:
            # for no_class in range(6,12):
            #     segmentation[segmentation==no_class] = 6
            segmentation_one_hot = keras.utils.to_categorical(segmentation, classes_count, dtype=int)

            images_for_batch.append(image)
            segmentation_for_batch.append(segmentation_one_hot)
            images_for_batch = np.asarray(images_for_batch)
            segmentation_for_batch = np.asarray(segmentation_for_batch)
        yield images_for_batch, segmentation_for_batch


In [ ]:
model = Unet('resnet34', input_shape=(352,480, 3), intencoder_weights='imagenet', classes=classes_count, activation='softmax')
model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#для сохранения весов каждые 10 эпох. обучаемся пока не остановим
while True:
    model.fit_generator(generator(), steps_per_epoch=256, epochs=10)
    model.save_weights(...)

In [ ]:
preprocess_input = get_preprocessing(BACKBONE) # функция segmentation_models
BACKBONE = 'resnet34'

In [ ]:
#генератор для выбора картинок из файла
def base_datagen(images_size,batch_size,im_path,lbl_path,s):
  
  image_list = glob(im_path)
  label_list = glob(lbl_path)
  size = len(image_list)//s
  image_list = image_list[size:]
  label_list = label_list[size:]
  indx = np.arange(len(image_list))
  np.random.shuffle(indx)
  image_list = np.asarray(image_list)[indx]
  label_list = np.asarray(label_list)[indx]
  while True :
    batch_x = []
    batch_y = []
    t = cycle(zip(image_list,label_list))
    for i in range(batch_size):
      im,lab = next(t)
      img = image.load_img(im,target_size = images_size)
      img = image.img_to_array(img)
      img = np.expand_dims(img,axis = 0)
      img = preprocess_input(img)
      batch_x.append(img)
      
      
      tar = image.load_img(lab,target_size = images_size)
      tar = image.img_to_array(tar)
      tar = np.expand_dims(tar,axis = 0)
      tar = tar[:,:,:,0]
      #tar[tar>6]=0
      tar = utils.to_categorical(tar,12)#7
     # tar = preprocess_input(tar)
      batch_y.append(tar)
    x = np.vstack(batch_x)
    y = np.vstack(batch_y)
    
    yield x,y

In [ ]:
#генератор для валидационной части
def val_gen(images_size,batch_size,im_path,lbl_path,s):
  image_list = glob(im_path)
  label_list = glob(lbl_path)
  size = len(image_list)//s
  image_list = image_list[0:size]
  label_list = label_list[0:size]
  indx = np.arange(len(image_list))
  #np.random.shuffle(indx)
  image_list = np.asarray(image_list)[indx]
  label_list = np.asarray(label_list)[indx]
  while True :
    batch_x_val = []
    batch_y_val = []
    t = cycle(zip(image_list,label_list))
    for i in range(batch_size):
      im_val,lab_val = next(t)
      img_val = image.load_img(im_val,target_size = images_size)
      img_val = image.img_to_array(img_val)
      img_val = np.expand_dims(img_val,axis = 0)
     # img_val = preprocess_input(img_val)
      batch_x_val.append(img_val)
      
      
      tar_val = image.load_img(lab_val,target_size = images_size)
      tar_val = image.img_to_array(tar_val)
      tar_val = np.expand_dims(tar_val,axis = 0)
      tar_val = tar_val[:,:,:,0]
      #tar_val[tar_val>6]=0
      tar_val = utils.to_categorical(tar_val,12+1)#7
     # tar = preprocess_input(tar)
      batch_y_val.append(tar_val)
    x_val = np.vstack(batch_x_val)
    y_val = np.vstack(batch_y_val)
    
    yield x_val,y_val

In [ ]:
#генератор для подачи тестовой выборки из файла
def test_gen(test_im_path,batch_size2,images_size):
  image_list = glob.glob(test_im_path) 
  indx = np.arange(len(image_list))
  image_list = np.asarray(image_list)[indx]
  while True :
    batch_x = []
    t = cycle(image_list)
    for i in range(batch_size):
      im = next(t)
      img = image.load_img(im,target_size = images_size)
      img = image.img_to_array(img)
      img = np.expand_dims(img,axis = 0)
      img = preprocess_input(img)
      batch_x.append(img)
    x = np.vstack(batch_x)
    yield x
    

In [ ]:
##### подача аугментированых картинок и их масок из файлов
#вспомогательная функция для кодирования аугментированных картинок в классы как у исходной картинки
def aug_coder(y_):
  dim = y_.shape
  y_new = np.zeros(y_.shape[:2], dtype=np.int32)
  for i, j in product(range(dim[0]), range(dim[1])):
     y_new[i][j] = y_[i][j][0]
  return y_new

In [ ]:
#определяем как будем изменять картинки
seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5),      # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

In [ ]:
# функция аугментации(получаем батч аугментированных картинок и их меток классов на каждую поданную пару)
def aug_batch(x_t, y_t, batch,classes):
  img_arr = []
  segmap_arr = []
  for i in range(batch):
    im  = x_t[0]
    segmap = aug_coder(y_t[0])
    segmap = ia.SegmentationMapOnImage(segmap, shape=im.shape, nb_classes=classes+1)#доп класс фон
    seq_det = seq.to_deterministic()
    img_aug = seq_det.augment_image(im)
    img_aug = np.expand_dims(img_aug,axis = 0)
    
    seg_aug = seq_det.augment_segmentation_maps([segmap])[0]
 #   segmap_aug = seg_aug.draw(size=img_aug.shape[:2]) # переводит размеченную  картинку в вид для изображения
    
    seg_map = seg_aug.get_arr_int()
    seg_map = utils.to_categorical(seg_map[:,:],classes+1)
    seg_map = np.expand_dims(seg_map,axis = 0)
   
        
  # show_all(x_t[i], img_aug,color_encode(y_t[i], colors), segmap_aug)
   #cv2_imshow(img_aug)
   #cv2_imshow(segmap_aug)
    img_arr.append(img_aug)
    segmap_arr.append(seg_map)
  return img_arr, segmap_arr

In [ ]:
#собственно сам генератора аугментированных картинок
def aug_im_gen(images_size,batch_size,im_path,lbl_path,s):
  
  image_list =[]
  label_list =[]  
  image_list = glob(im_path)
  label_list = glob(lbl_path)
  size = len(image_list)//s
  image_list = image_list[size:]
  label_list = label_list[size:]
  indx = np.arange(len(image_list))
  np.random.shuffle(indx)
  image_list = np.asarray(image_list)[indx]
  label_list = np.asarray(label_list)[indx]
  while True :
    x_b = []
    y_b = []
    t = cycle(zip(image_list,label_list))
    
    im,lab = next(t)
    img = image.load_img(im,target_size = images_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis = 0)
   # img = preprocess_input(img)
    x_b = img
        
    tar = image.load_img(lab,target_size = images_size)
    tar = image.img_to_array(tar)
    tar = np.expand_dims(tar,axis = 0)  
    y_b = tar
    
    batch_x = []
    batch_y = [] 
    batch_x,batch_y = aug_batch(x_b, y_b, batch_size,12)
   
    x = np.vstack(batch_x)
    y = np.vstack(batch_y)
    
    yield x,y

In [ ]:
palette = {0:(30,50,70),1:(0,140,55),2:(70,222,70),3:(33,55,150),4:(220,127,45),5:(100,40,80),6:(100,0,0)}

In [ ]:
#Демонстрация размеченного изображения в цвете
def colored_picture(img_path,palette):
  ar = image.load_image(img_path,target_size = (352,480))
  ar = image.img_to_array(ar)
  ar = ar[:,:,0]
  R = np.zeros(ar.shape)
  G = np.zeros(ar.shape)
  B = np.zeros(ar.shape)
  for cat in palette:
    R[arr==cat]=palette[cat][0]
    G[arr==cat]=palette[cat][1]
    B[arr==cat]=palette[cat][2]
  img = np.zeros(ar.shape)
  img = np.expand_dims(img, axis = 3)
  img = np.dstack((R,G,B))
  img = image.array_to_img(img)
  #plt.imshow(img)
  return img

In [ ]:
#Преобразование тензора на выходе сети в раскрашенную картинку
def colored_encode(x,palette):
  R = np.zeros(x[0].shape)
  G = np.zeros(x[0].shape)
  B = np.zeros(x[0].shape)
  for cat in palette:
    R[x[0]==cat] = palette[cat][0]
    G[x[0]==cat] = palette[cat][1]
    B[x[0]==cat] = palette[cat][2]
  img = np.zeros(x.shape)
  img = np.expand_dims(img, axis = 3)
  img = np.dstack((R,G,B))
  img = image.array_to_img(img)
  #plt.imshow(img)
  return img  